Transform Data

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras import models
import numpy as np 

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import pandas as pd

In [ ]:
#https://archive.ics.uci.edu/ml/machine-learning-databases/00298/?fbclid=IwAR1yHS5kSR-RNcSOT3Fd8crfh3z-YKNA4xYFhYHh76-qmVunZYpBCe5TXZA
data = pd.read_csv('/content/jsbach_chorals_harmony.data')
data = np.array(data.values[0:100])
print(data.shape)
print(data[:5])

(100, 17)
[['000106b_' 2 'YES' ' NO' ' NO' ' NO' 'YES' ' NO' ' NO' 'YES' ' NO'
  ' NO' ' NO' ' NO' 'E' 5 ' C_M']
 ['000106b_' 3 'YES' ' NO' ' NO' ' NO' 'YES' ' NO' ' NO' 'YES' ' NO'
  ' NO' ' NO' ' NO' 'E' 2 ' C_M']
 ['000106b_' 4 'YES' ' NO' ' NO' ' NO' ' NO' 'YES' ' NO' ' NO' ' NO'
  'YES' ' NO' ' NO' 'F' 3 ' F_M']
 ['000106b_' 5 'YES' ' NO' ' NO' ' NO' ' NO' 'YES' ' NO' ' NO' ' NO'
  'YES' ' NO' ' NO' 'F' 2 ' F_M']
 ['000106b_' 6 ' NO' ' NO' 'YES' ' NO' ' NO' 'YES' ' NO' ' NO' ' NO'
  'YES' ' NO' ' NO' 'D' 4 ' D_m']]


In [ ]:
X = data[:,:-1]
y = data[:,-1]
print(X.shape)
print(y.shape)

(100, 16)
(100,)


In [ ]:
from collections import Counter
counter = Counter()
counter.update(y)
print(counter)

Counter({' F_M': 27, ' BbM': 22, ' C_M7': 16, ' C_M': 13, ' D_m7': 6, ' C_M4': 6, ' D_m': 4, ' A_m': 4, ' G_M': 2})


In [ ]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for i in range(X.shape[1]):
    X[:,i] = encoder.fit_transform(X[:,i])
y = encoder.fit_transform(y)
X = np.asarray(X.astype(np.float32))
print(X)
print(y)

[[ 0.  0.  1. ...  0.  5.  4.]
 [ 0.  1.  1. ...  0.  5.  1.]
 [ 0.  2.  1. ...  0.  6.  2.]
 ...
 [ 0. 97.  1. ...  0.  5.  4.]
 [ 0. 98.  1. ...  0.  5.  1.]
 [ 0. 99.  1. ...  0.  5.  0.]]
[2 2 7 7 5 5 7 7 1 1 1 1 1 1 7 7 1 1 1 4 4 4 7 7 2 2 6 6 6 8 2 7 7 1 1 0 0
 4 4 7 7 3 3 3 4 4 4 7 7 2 2 7 7 5 5 7 7 1 1 1 1 1 1 7 7 1 1 1 4 4 4 7 7 2
 2 6 6 6 8 2 7 7 1 1 0 0 4 4 7 7 3 3 3 4 4 4 7 2 2 2]


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print('Train:')
print(X_train.shape)
print(y_train.shape)
print('Test:')
print(X_test.shape)
print(y_test.shape)

Train:
(70, 16)
(70,)
Test:
(30, 16)
(30,)


In [ ]:
from tensorflow.keras.utils import to_categorical
y_train = to_categorical(y_train)
print(y_train[:2])

[[0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0.]]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers , activations , models , preprocessing , utils
import tensorflow as tf
from  sklearn.model_selection import train_test_split

In [ ]:
print(X_train.shape[1])

16


# Model

In [ ]:
#embed_dim = 128
hidden_size = 200
batch_size = 32

timesteps = X_train.shape[1]
x_dim = 1

model = models.Sequential()
model.add(layers.SimpleRNN(hidden_size,input_shape=(timesteps,x_dim)))
model.add(layers.Dense(128,activation='sigmoid'))
model.add(layers.Dense(9,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_1 (SimpleRNN)    (None, 200)               40400     
                                                                 
 dense_3 (Dense)             (None, 128)               25728     
                                                                 
 dense_4 (Dense)             (None, 9)                 1161      
                                                                 
Total params: 67,289
Trainable params: 67,289
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
#reshape 
a = np.array([[1,2,3],[4,5,6]])
print(a)
print(a.shape)
b = a.reshape(2,3,1)
print(b)

[[1 2 3]
 [4 5 6]]
(2, 3)
[[[1]
  [2]
  [3]]

 [[4]
  [5]
  [6]]]


In [ ]:
# chỉnh lại đầu vào 
print(X_train.shape)
n_examples = X_train.shape[0]
time_steps = X_train.shape[1] 
X_train_transformed = X_train.reshape(n_examples,time_steps,1)
print(X_train_transformed.shape)

(70, 16)
(70, 16, 1)


In [ ]:
model.fit(X_train_transformed, y_train, batch_size = 30, epochs = 10)

Epoch 1/10
3/3 [==============================] - 1s 11ms/step - loss: 2.1850 - accuracy: 0.1714
Epoch 2/10
3/3 [==============================] - 0s 10ms/step - loss: 1.9133 - accuracy: 0.2571
Epoch 3/10
3/3 [==============================] - 0s 11ms/step - loss: 1.8615 - accuracy: 0.2000
Epoch 4/10
3/3 [==============================] - 0s 14ms/step - loss: 1.7346 - accuracy: 0.4143
Epoch 5/10
3/3 [==============================] - 0s 17ms/step - loss: 1.6808 - accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 0s 11ms/step - loss: 1.5849 - accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 10ms/step - loss: 1.5136 - accuracy: 0.5429
Epoch 8/10
3/3 [==============================] - 0s 11ms/step - loss: 1.3837 - accuracy: 0.6143
Epoch 9/10
3/3 [==============================] - 0s 10ms/step - loss: 1.2984 - accuracy: 0.6286
Epoch 10/10
3/3 [==============================] - 0s 10ms/step - loss: 1.2017 - accuracy: 0.6714


In [ ]:
X_test_transformed = X_test.reshape(len(X_test),time_steps,1)
y_pred = model.predict(X_test_transformed)
print(y_pred[:2])

[[0.01036327 0.7841053  0.01480247 0.0332904  0.0462744  0.01855203
  0.01209651 0.06793924 0.01257625]
 [0.01877417 0.30525962 0.02340603 0.0304589  0.04063199 0.03723297
  0.05257177 0.47273368 0.01893089]]


In [ ]:
y_pred = y_pred.argmax(axis=1)
print(y_pred)
print(y_test)

[1 7 2 4 4 7 1 7 1 2 1 2 2 1 1 7 7 7 1 7 7 7 7 7 2 7 7 7 1 1]
[1 5 4 4 4 7 7 7 1 2 1 2 2 1 3 5 6 6 1 7 7 7 6 3 4 7 7 7 1 7]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           1       0.67      1.00      0.80         6
           2       0.60      1.00      0.75         3
           3       0.00      0.00      0.00         2
           4       1.00      0.50      0.67         4
           5       0.00      0.00      0.00         2
           6       0.00      0.00      0.00         3
           7       0.57      0.80      0.67        10

    accuracy                           0.63        30
   macro avg       0.41      0.47      0.41        30
weighted avg       0.52      0.63      0.55        30



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1308: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
